(sec:pyalf-check_warmup)=
# Checking warmup and autocorrelation times

Two common challenges in Monte Carlo studies are ensuring that the measured bins represent **equilibrated** configurations and that different bins are **statistically independent**. In this section, we will briefly explain these issues and present the tools pyALF offers for dealing with them.

## Preparations

As a first step, we use the same import as in previous sections.

In [1]:
# Enable Matplotlib Jupyter Widget Backend
%matplotlib widget

from py_alf.utils import find_sim_dirs  # Function for finding QMC bins

In [2]:
# Change some Matplotlib defaults for nicer figures
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [5, 3]
mpl.rcParams["font.family"] = "serif"
mpl.rcParams['font.size'] = 8
mpl.rcParams["xtick.direction"] = 'in'
mpl.rcParams["ytick.direction"] = 'in'
mpl.rcParams["xtick.top"] = True
mpl.rcParams["ytick.right"] = True
mpl.rcParams["figure.constrained_layout.use"] = True

We also import the functions {func}`py_alf.check_warmup` and {func}`py_alf.check_rebin`, which play the main role in this section.

In [3]:
from py_alf import check_rebin, check_warmup

Finally, from the local file `custom_obs.py`, we import the same `custom_obs` defined in the previous section.

In [4]:
from custom_obs import custom_obs

For demonstration purposes, we run a simulation with very small bins.

In [5]:
from py_alf import ALF_source, Simulation

sim = Simulation(
    ALF_source(),
    'Hubbard',
    {
        # Model specific parameters
        'L1': 4,
        'L2': 4,
        'Ham_U': 5.0,
        # QMC parameters
        'Nbin': 5000,
        'Nsweep': 5,
        'Ltau': 0,
    },
)
sim.compile()
sim.run()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


entanglement_mod.F90:35:2:

   35 | #warning "You are compiling entanglement without MPI. No Renyi entropy results possible, all other observables still work!"
      |  1~~~~~~
ar: creating modules_90.a
ar: creating libqrref.a


Compiling Analysis
Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules
Link program
Done.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=

ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
Compiling program modules


Link program


Done.
Prepare directory "/home/jonas/Programs/pyALF/doc/source/usage/ALF_data/Hubbard_L1=4_L2=4_U=5.0" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


We set the directories to be considered.

In [6]:
dirs = find_sim_dirs()
dirs

['./ALF_data/Hubbard',
 './ALF_data/Hubbard_L1=4_L2=4_U=1.0',
 './ALF_data/Hubbard_L1=4_L2=4_U=2.0',
 './ALF_data/Hubbard_L1=4_L2=4_U=3.0',
 './ALF_data/Hubbard_L1=4_L2=4_U=4.0',
 './ALF_data/Hubbard_L1=4_L2=4_U=5.0',
 './ALF_data/Hubbard_Square',
 './ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5/Temp_0',
 './ALF_data/temper_Hubbard_L1=4_L2=4_U=2.5/Temp_1']

## Check warmup

A Monte Carlo simulation creates a time series of configurations through stochastic updates. Usually, measurements from a number of updates get combined in one so-called bin. In the case of ALF, `Nsweep` sweeps create one bin of measurements (for more details on updating procedures we refer to the [ALF documentation](https://git.physik.uni-wuerzburg.de/ALF/ALF/-/jobs/artifacts/master/raw/Documentation/doc.pdf?job=create_doc)). Usually, the simulation starts in a non-optimal state and it takes some time to reach equilibrium. Bins from this "warming up" period should be dismissed before calculating results. This is achieved by setting the variable `N_skip` in the file `parameters`, which will make the analysis omit the first `N_skip` bins.

```{warning}
Different observables can have different warmup and autocorrelation times. For example, charge degrees of freedom may equilibrate much faster than spin degrees of freedom. Or a sum of observables might have much shorter autocorrelation times than an individual observable, e.g. the total spin versus one spin component.
```

To judge the correct value for `N_skip`, pyALF offers the function {func}`check_warmup() <py_alf.check_warmup>`, which plots the time series of bins for a given list of scalar and custom observables. It can be used with the previous simulations as:

In [7]:
warmup_widget = check_warmup(
    dirs,
    ['Ener_scal', 'Kin_scal', 'Pot_scal',
     'E_pot_kin', 'R_Ferro', 'R_AFM',
     'SpinZ_pipi', 'SpinXY_pipi', 'SpinXYZ_pipi'],
    custom_obs=custom_obs, gui='ipy'
)

The first argument is a list of directories containing simulations, the second argument specifies which observables to plot, the keyword argument `custom_obs` is needed, when plotting custom observables, e.g. *E_pot_kin* and `gui` specifies which GUI framework to use. With `gui='ipy'`, the function returns a [Jupyter Widget](https://ipywidgets.readthedocs.io), which allows to seamlessly work within Jupyter. Another option would be `gui='tk'`, which opens a separate window using {mod}`tkinter`. The latter option might be suitable when working directly from a shell.

The variable `N_skip` can be directly changed in the GUI, which automatically updates the file `parameters`.

In [8]:
warmup_widget

## Check rebin

When estimating statistical errors, the analysis assumes different bins to be statistically independent. As a result, one bin must span over enough updates to generate statically independent configurations, or in other words, a bin must be larger than the autocorrelation time. Otherwise the statistical errors will be underestimated. To address this issue, the analysis employs so-called rebinning, which combines `N_rebin` bins into one new bin. The pyALF function {func}`check_rebin() <py_alf.check_rebin>` helps in determining the correct `N_rebin`. It plots the errors of the chosen observables against `N_rebin`. With enough statistics, one should see growing errors with increasing `N_rebin` until a saturation point is reached, this saturation point marks a suitable value for `N_rebin`. The usage of {func}`check_rebin() <py_alf.check_rebin>` is very similar to {func}`check_warmup() <py_alf.check_warmup>`.

In [9]:
rebin_widget = check_rebin(
    dirs,
    ['Ener_scal', 'Kin_scal', 'Pot_scal',
     'E_pot_kin', 'R_Ferro', 'R_AFM',
     'SpinZ_pipi', 'SpinXY_pipi', 'SpinXYZ_pipi'],
    custom_obs=custom_obs, gui='ipy'
)

Below, we can see how different observables have different autocorrelation times. While the error of the kinetic energy saturates already at $N_\text{rebin} = 3$, the correlations of the z component of the spin at $(\pi, \pi)$ (`SpinZ_pipi`) need $N_\text{rebin} \sim 40$. For the correlations of the total spin (`SpinXYZ_pipi`), on the other hand, $N_\text{rebin} = 1$ is sufficient.

The improvement from `SpinZ_pipi` to `SpinXYZ_pipi` is a good example for the concept of an improved estimator: The simulated Hubbard model is $\text{SU}(2)$ symmetric, therefore correlations of the x, y and z components of the spin are equivalent, but with the chosen parameter `Mz=True` (cf. {numref}`seq:pyalf_comprun`) the auxiliary field couples to the z component of the spin. As a result, the $\text{SU}(2)$ symmetry is broken for an individual auxiliary field configuration, but restored by sampling the field configurations. 
Therefore, measuring the spin correlations through the z component, the x-y plane, or the full spin are in principle equivalent. But the latter option produces the most precise results and has the shortest autocorrelation times, because it explicitly restores the $\text{SU}(2)$ symmetry instead of "waiting" for the sampling to do that.

Furthermore, the ferromagnetic correlation ratio $R_{\text{Ferro}}$ doesn't seem to converge at all in the considered range of $N_{\text{rebin}}$. This is connected to the fact that the system is not close the ferromagnetic order and therefore $R_{\text{Ferro}}$ is a bad observable.

In [10]:
rebin_widget

The next section will also show options for an improved estimator by employing symmetry operations on the Bravais lattice.